In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import sys
import os
from torchvision.transforms import v2
import torch
import shutil
from PIL import Image
from ultralytics import YOLO
import cv2
import pandas as pd
from PIL import Image
import random
import albumentations as A
from albumentations.pytorch import ToTensorV2
import numpy as np
from IPython.display import clear_output
import yaml
import os


# Local dep
project_dir = '/data/konrad/workspace'
sys.path.insert(0, project_dir)

from helpers.datasets import CalfCenterFaceDataset
from helpers.helpers import get_indices, uniform_sample_with_values, load_face_data

INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.14 (you have 1.4.12). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.


In [2]:
def plot_bbox_gallery(image_df, n_cols=5):
    n_images = image_df.shape[0]
    n_rows = n_images // n_cols + int(n_images % n_cols > 0)
    
    plt.figure(figsize=(20, n_rows * 4))
    for i, row in image_df.iterrows():
        img = Image.open(row["path"])
        # plt.subplot(n_rows, n_cols, i + 1)
        ax = plt.subplot(n_rows, n_cols, i + 1)
        plt.imshow(img)
        # ax = plt.gca()
        score = row["conf"]
        box_x = row["box_x"]
        box_y = row["box_y"]
        box_width = row["box_width"]
        box_height = row["box_height"]
        box_x = box_x - (box_width / 2)
        box_y = box_y - (box_height / 2)
        rect = plt.Rectangle((box_x, box_y), box_width, box_height, edgecolor='r', facecolor='none')
        ax.add_patch(rect)
        plt.text(box_x, box_y - 10, f' {score:.2f}', color='red', fontsize=12, backgroundcolor='white')
        plt.axis('off')
    
    plt.tight_layout()
    plt.show()


def apply_transformations(image, bbox, transforms, label_id):
    transformed = transforms(image=image, bboxes=[bbox], category_ids=[label_id])
    
    transformed_image = transformed['image']
    transformed_bbox = transformed['bboxes'][0]
    
    return transformed_image, transformed_bbox
    
def df_to_yolo(df, root_dir, transforms = None, num_gen=3, img_src_dir = None):
    
    img_path = root_dir + "/" + "images"
    if not os.path.exists(img_path):
        os.makedirs(img_path)

    lbl_path = root_dir + "/" + "labels"
    if not os.path.exists(lbl_path):
        os.makedirs(lbl_path)

    for index, row in df.iterrows():
        # Extract the image name
        image_name = row['image']

        img_src_dir = row['path']
        
        class_idx = row['target']

        xmin = row['x_min']
        ymin = row['y_min']
        xmax = row['x_max']
        ymax = row['y_max']

        bbox = [xmin, ymin, xmax, ymax]
        
        if transforms is None:

            shutil.copy(os.path.join(img_src_dir), os.path.join(img_path, image_name))
            filename = os.path.splitext(image_name)[0]

            # Open the image file
            with Image.open(os.path.join(img_src_dir)) as img:
            # Get the width and height
                imgWidth, imgHeight = img.size

            center_x = (xmin + xmax) / 2.0
            center_y = (ymin + ymax) / 2.0
    
            width = xmax - xmin
            height = ymax - ymin
    
            norm_center_x = center_x / imgWidth
            norm_center_y = center_y / imgHeight
            
            norm_width = width / imgWidth
            norm_height = height / imgHeight
            
            box_annotation = ' '.join([str(class_idx), str(norm_center_x), str(norm_center_y), str(norm_width), str(norm_height)])+'\n'
    
            label_filename = lbl_path + '/' + filename + ".txt"
            anno_f = open(label_filename, 'w')
            anno_f.writelines(box_annotation)
            anno_f.close()

            continue

        original_image_path = os.path.join(img_src_dir)
        img = cv2.imread(original_image_path)
        imgHeight, imgWidth, _ = img.shape
        
        for i in range(num_gen):  # Apply transformations 5 times
            transformed_image, transformed_bbox = apply_transformations(img, bbox, transforms, class_idx)
            
            center_x = (transformed_bbox[0] + transformed_bbox[2]) / 2.0
            center_y = (transformed_bbox[1] + transformed_bbox[3]) / 2.0

            width = transformed_bbox[2] - transformed_bbox[0]
            height = transformed_bbox[3] - transformed_bbox[1]
            
            norm_center_x = center_x / imgWidth
            norm_center_y = center_y / imgHeight
            
            norm_width = width / imgWidth
            norm_height = height / imgHeight
            
            box_annotation = ' '.join([str(class_idx), str(norm_center_x), str(norm_center_y), str(norm_width), str(norm_height)]) + '\n'
            
            transformed_image_name = f"{os.path.splitext(image_name)[0]}_transformed_{i}.jpg"
            cv2.imwrite(os.path.join(img_path, transformed_image_name), transformed_image)
            
            label_filename = os.path.join(lbl_path, f"{os.path.splitext(transformed_image_name)[0]}.txt")
            with open(label_filename, 'w') as anno_f:
                anno_f.writelines(box_annotation)

    print("Done !")


def delete_dir_if_exists(dir_path):
    if os.path.exists(dir_path):
        try:
            shutil.rmtree(dir_path)
            print(f"Directory '{dir_path}' deleted")
        except OSError as e:
            print(f"Error deleting directory '{dir_path}': {e}")

def set_yml(file_name, data_path, id2label, train_dir = "train", test_dir = "test", val_dir = "val"):
    
    # Create the dictionary structure for YAML
    yolo_detect_config = {
        'path': data_path,
        'train': train_dir,
        'test': test_dir,
        'val': val_dir,
        'names': id2label
    }
    
    # Define the output file path
    yml_file_path = os.path.join(data_path, f"{file_name}.yml")
    
    # Write the YAML file
    with open(yml_file_path, 'w') as yml_file:
        yaml.dump(yolo_detect_config, yml_file, default_flow_style=False)
        
    print("Yml file set !")
    return yml_file_path

In [3]:
ROOT_DIR = "/data/konrad/workspace"

dataset_type = "splt"
label_col = "label"

num_labels = 3 if label_col == "label" else 1
train_df = pd.read_csv(ROOT_DIR + f'/csv_files/mixed_10s_b0s_y7_1/{dataset_type}_train_image_extracted_metadata.csv', index_col=False)
valid_df = pd.read_csv(ROOT_DIR + f'/csv_files/mixed_10s_b0s_y7_1/{dataset_type}_test_image_extracted_metadata.csv', index_col=False)
test_df = pd.read_csv(ROOT_DIR + f'/csv_files/mixed_10s_b0s_y7_1/{dataset_type}_val_image_extracted_metadata.csv', index_col=False)
labels = train_df[label_col].unique()
label2id = {l:i for i, l in enumerate(labels)}
id2label = {i:l for i, l in enumerate(labels)}
train_df['target'] = train_df.apply(lambda row: label2id[row[label_col]], axis=1)
test_df['target'] = test_df.apply(lambda row: label2id[row[label_col]], axis=1)
valid_df['target'] = valid_df.apply(lambda row: label2id[row[label_col]], axis=1)
# test_df = test_df.sample(frac=.10).reset_index(drop=True)

IMAGE_DIR = ROOT_DIR + f"/datasets/yolo_detection_{dataset_type}"
num_gen = 1

In [4]:
transforms = A.Compose([
    A.Rotate(limit=(10, 20), p=.7),
    A.OneOf([
        A.GaussianBlur(blur_limit=(3,9), p=.7),
        A.MedianBlur(blur_limit=7, p=.7),
    ], p=.6),
    A.Sharpen(p=.7),
], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['category_ids']))

delete_dir_if_exists(IMAGE_DIR)
df_to_yolo(train_df, IMAGE_DIR + '/train', img_src_dir = IMAGE_DIR, transforms = transforms, num_gen=num_gen)
# df_to_yolo(train_df, IMAGE_DIR + '/train', img_src_dir = IMAGE_DIR, transforms = None, num_gen=num_gen)
df_to_yolo(valid_df, IMAGE_DIR + '/test', img_src_dir = IMAGE_DIR)
df_to_yolo(test_df, IMAGE_DIR + '/val', img_src_dir = IMAGE_DIR)

# Load a model
model = YOLO("yolov8m.pt")  # load a pretrained model (recommended for training)
epoch = 10

model_name = "yolo_detect"
yml_file_path = set_yml(f"{model_name}_{dataset_type}", IMAGE_DIR, id2label)
results = model.train(data=yml_file_path, epochs=epoch, project=ROOT_DIR + "/training_log", name=model_name, cfg="config.yml")
# results = model.train(data="yolo_detect.yml", epochs=epoch, project=ROOT_DIR + "/training_log", name="yolo_detect")
results = model.val(data=yml_file_path, split='test', max_det=1, conf=.80)  # Evaluate on test set

model.export()

Directory '/data/konrad/workspace/datasets/yolo_detection_splt' deleted
Done !
Done !
Done !
Yml file set !
New https://pypi.org/project/ultralytics/8.2.84 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.58 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4090, 24115MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/data/konrad/workspace/datasets/yolo_detection_splt/yolo_detect_splt.yml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/data/konrad/workspace/training_log, name=yolo_detect8, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_de

train: Scanning /data/konrad/workspace/datasets/yolo_detection_splt/train/labels... 5497 images, 0 backgrounds, 0 corrup


train: New cache created: /data/konrad/workspace/datasets/yolo_detection_splt/train/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


val: Scanning /data/konrad/workspace/datasets/yolo_detection_splt/val/labels... 1379 images, 0 backgrounds, 0 corrupt: 1


val: New cache created: /data/konrad/workspace/datasets/yolo_detection_splt/val/labels.cache
Plotting labels to /data/konrad/workspace/training_log/yolo_detect8/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001429, momentum=0.9) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.0005), 83 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to /data/konrad/workspace/training_log/yolo_detect8
Starting training for 10 epochs...
Closing dataloader mosaic
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

val: Scanning /data/konrad/workspace/datasets/yolo_detection_splt/test/labels... 283 images, 0 backgrounds, 0 corrupt: 1

val: New cache created: /data/konrad/workspace/datasets/yolo_detection_splt/test/labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:02<


                   all        283        283      0.503      0.175      0.322      0.269
              Diarrhea         99         99      0.828      0.242      0.539      0.455
             Pneumonia         75         75       0.13       0.08     0.0712     0.0628
               Healthy        109        109       0.55      0.202      0.356       0.29
Speed: 0.1ms preprocess, 2.1ms inference, 0.0ms loss, 1.5ms postprocess per image
Results saved to /data/konrad/workspace/training_log/yolo_detect82
Ultralytics YOLOv8.2.58 🚀 Python-3.10.12 torch-2.3.1+cu121 CPU (AMD Ryzen Threadripper PRO 5975WX 32-Cores)

PyTorch: starting from '/data/konrad/workspace/training_log/yolo_detect8/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 7, 8400) (49.6 MB)

TorchScript: starting export with torch 2.3.1+cu121...
TorchScript: export success ✅ 1.2s, saved as '/data/konrad/workspace/training_log/yolo_detect8/weights/best.torchscript' (99.1 MB)

Export complete (2.7s)
Res

'/data/konrad/workspace/training_log/yolo_detect8/weights/best.torchscript'

In [6]:
transforms = A.Compose([
    A.Rotate(limit=(10, 20), p=.7),
    A.OneOf([
        A.GaussianBlur(blur_limit=(3,9), p=.7),
        A.MedianBlur(blur_limit=7, p=.7),
    ], p=.6),
    A.Sharpen(p=.7),
], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['category_ids']))

delete_dir_if_exists(IMAGE_DIR)
df_to_yolo(train_df, IMAGE_DIR + '/train', img_src_dir = IMAGE_DIR, transforms = transforms, num_gen=num_gen)
# df_to_yolo(train_df, IMAGE_DIR + '/train', img_src_dir = IMAGE_DIR, transforms = None, num_gen=num_gen)
df_to_yolo(valid_df, IMAGE_DIR + '/val', img_src_dir = IMAGE_DIR)
df_to_yolo(test_df, IMAGE_DIR + '/test', img_src_dir = IMAGE_DIR)

# Load a model
model = YOLO("yolov8m.pt")  # load a pretrained model (recommended for training)
epoch = 10

model_name = "yolo_detect"
yml_file_path = set_yml(f"{model_name}_{dataset_type}", IMAGE_DIR, id2label)
results = model.train(data=yml_file_path, epochs=epoch, project=ROOT_DIR + "/training_log", name=model_name, cfg="config.yml")
# results = model.train(data="yolo_detect.yml", epochs=epoch, project=ROOT_DIR + "/training_log", name="yolo_detect")
# results = model.val(data=yml_file_path, split='test', max_det=1, conf=.80)  # Evaluate on test set

model.export()

Directory '/data/konrad/workspace/datasets/yolo_detection_splt' deleted
Done !
Done !
Done !
Yml file set !
New https://pypi.org/project/ultralytics/8.2.84 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.58 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4090, 24115MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/data/konrad/workspace/datasets/yolo_detection_splt/yolo_detect_splt.yml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/data/konrad/workspace/training_log, name=yolo_detect9, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_de

train: Scanning /data/konrad/workspace/datasets/yolo_detection_splt/train/labels... 5497 images, 0 backgrounds, 0 corrup


train: New cache created: /data/konrad/workspace/datasets/yolo_detection_splt/train/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


val: Scanning /data/konrad/workspace/datasets/yolo_detection_splt/val/labels... 283 images, 0 backgrounds, 0 corrupt: 10


val: New cache created: /data/konrad/workspace/datasets/yolo_detection_splt/val/labels.cache
Plotting labels to /data/konrad/workspace/training_log/yolo_detect9/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001429, momentum=0.9) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.0005), 83 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to /data/konrad/workspace/training_log/yolo_detect9
Starting training for 10 epochs...
Closing dataloader mosaic
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        283        283      0.374      0.656      0.432      0.335



10 epochs completed in 0.147 hours.
Optimizer stripped from /data/konrad/workspace/training_log/yolo_detect9/weights/last.pt, 52.0MB
Optimizer stripped from /data/konrad/workspace/training_log/yolo_detect9/weights/best.pt, 52.0MB

Validating /data/konrad/workspace/training_log/yolo_detect9/weights/best.pt...
Ultralytics YOLOv8.2.58 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4090, 24115MiB)
Model summary (fused): 218 layers, 25,841,497 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:01<00


                   all        283        283      0.409      0.654      0.519      0.379
              Diarrhea         99         99      0.376      0.899        0.6      0.445
             Pneumonia         75         75      0.345      0.347      0.322      0.235
               Healthy        109        109      0.505      0.716      0.634      0.458
Speed: 0.1ms preprocess, 0.8ms inference, 0.0ms loss, 2.5ms postprocess per image
Results saved to /data/konrad/workspace/training_log/yolo_detect9
Ultralytics YOLOv8.2.58 🚀 Python-3.10.12 torch-2.3.1+cu121 CPU (AMD Ryzen Threadripper PRO 5975WX 32-Cores)
Model summary (fused): 218 layers, 25,841,497 parameters, 0 gradients, 78.7 GFLOPs

PyTorch: starting from '/data/konrad/workspace/training_log/yolo_detect9/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 7, 8400) (49.6 MB)

TorchScript: starting export with torch 2.3.1+cu121...
TorchScript: export success ✅ 1.3s, saved as '/data/konrad/workspace/trainin

'/data/konrad/workspace/training_log/yolo_detect9/weights/best.torchscript'

In [4]:
transforms = A.Compose([
    A.Rotate(limit=(10, 20), p=.7),
    A.OneOf([
        A.GaussianBlur(blur_limit=(3,9), p=.7),
        A.MedianBlur(blur_limit=7, p=.7),
    ], p=.6),
    A.Sharpen(p=.7),
], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['category_ids']))


delete_dir_if_exists(IMAGE_DIR)
# df_to_yolo(train_df, IMAGE_DIR + '/train', img_src_dir = IMAGE_DIR, transforms = transforms, num_gen=num_gen)
df_to_yolo(train_df, IMAGE_DIR + '/train', img_src_dir = IMAGE_DIR, transforms = None, num_gen=num_gen)
df_to_yolo(valid_df, IMAGE_DIR + '/test', img_src_dir = IMAGE_DIR)
df_to_yolo(test_df, IMAGE_DIR + '/val', img_src_dir = IMAGE_DIR)

# Load a model
model = YOLO("yolov8m.pt")  # load a pretrained model (recommended for training)
epoch = 10

model_name = "yolo_detect"
yml_file_path = set_yml(f"{model_name}_{dataset_type}", IMAGE_DIR, id2label)
# results = model.train(data=yml_file_path, epochs=epoch, project=ROOT_DIR + "/training_log", name="yolo_detect", cfg="config.yml")
results = model.train(data=yml_file_path, epochs=epoch, project=ROOT_DIR + "/training_log", name=model_name)
results = model.val(data=yml_file_path, split='test')  # Evaluate on test set

model.export()

Directory '/data/konrad/workspace/datasets/yolo_detection_splt' deleted
Done !
Done !
Done !
Yml file set !
New https://pypi.org/project/ultralytics/8.2.83 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.58 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4090, 24115MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/data/konrad/workspace/datasets/yolo_detection_splt/yolo_detect_splt.yml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/data/konrad/workspace/training_log, name=yolo_detect6, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_de

train: Scanning /data/konrad/workspace/datasets/yolo_detection_splt/train/labels... 5497 images, 0 backgrounds, 0 corrup


train: New cache created: /data/konrad/workspace/datasets/yolo_detection_splt/train/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


val: Scanning /data/konrad/workspace/datasets/yolo_detection_splt/val/labels... 1379 images, 0 backgrounds, 0 corrupt: 1

val: New cache created: /data/konrad/workspace/datasets/yolo_detection_splt/val/labels.cache


...

val: Scanning /data/konrad/workspace/datasets/yolo_detection_splt/test/labels... 283 images, 0 backgrounds, 0 corrupt: 1


val: New cache created: /data/konrad/workspace/datasets/yolo_detection_splt/test/labels.cache


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:02<


                   all        283        283      0.356       0.82      0.429      0.332
              Diarrhea         99         99      0.358      0.798      0.584      0.459
             Pneumonia         75         75      0.286      0.773      0.267      0.202
               Healthy        109        109      0.425       0.89      0.436      0.334
Speed: 0.1ms preprocess, 2.0ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to /data/konrad/workspace/training_log/yolo_detect62
Ultralytics YOLOv8.2.58 🚀 Python-3.10.12 torch-2.3.1+cu121 CPU (AMD Ryzen Threadripper PRO 5975WX 32-Cores)

PyTorch: starting from '/data/konrad/workspace/training_log/yolo_detect6/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 7, 8400) (49.6 MB)

TorchScript: starting export with torch 2.3.1+cu121...
TorchScript: export success ✅ 1.2s, saved as '/data/konrad/workspace/training_log/yolo_detect6/weights/best.torchscript' (99.1 MB)

Export complete (2.7s)
Res

'/data/konrad/workspace/training_log/yolo_detect6/weights/best.torchscript'

In [6]:
transforms = A.Compose([
    A.Rotate(limit=(10, 20), p=.7),
    A.OneOf([
        A.GaussianBlur(blur_limit=(3,9), p=.7),
        A.MedianBlur(blur_limit=7, p=.7),
    ], p=.6),
    A.Sharpen(p=.7),
], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['category_ids']))

delete_dir_if_exists(IMAGE_DIR)
df_to_yolo(train_df, IMAGE_DIR + '/train', img_src_dir = IMAGE_DIR, transforms = transforms, num_gen=num_gen)
# df_to_yolo(train_df, IMAGE_DIR + '/train', img_src_dir = IMAGE_DIR, transforms = None, num_gen=num_gen)
df_to_yolo(valid_df, IMAGE_DIR + '/test', img_src_dir = IMAGE_DIR)
df_to_yolo(test_df, IMAGE_DIR + '/val', img_src_dir = IMAGE_DIR)

# Load a model
model = YOLO("yolov8m.pt")  # load a pretrained model (recommended for training)
epoch = 10

model_name = "yolo_detect"
yml_file_path = set_yml(f"{model_name}_{dataset_type}", IMAGE_DIR, id2label)
results = model.train(data=yml_file_path, epochs=epoch, project=ROOT_DIR + "/training_log", name=model_name, cfg="config.yml")
# results = model.train(data="yolo_detect.yml", epochs=epoch, project=ROOT_DIR + "/training_log", name="yolo_detect")
results = model.val(data=yml_file_path, split='test')  # Evaluate on test set

model.export()

Directory '/data/konrad/workspace/datasets/yolo_detection_splt' deleted
Done !
Done !
Done !
Yml file set !
New https://pypi.org/project/ultralytics/8.2.83 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.58 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4090, 24115MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/data/konrad/workspace/datasets/yolo_detection_splt/yolo_detect_splt.yml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/data/konrad/workspace/training_log, name=yolo_detect7, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_de

train: Scanning /data/konrad/workspace/datasets/yolo_detection_splt/train/labels... 5497 images, 0 backgrounds, 0 corrup


train: New cache created: /data/konrad/workspace/datasets/yolo_detection_splt/train/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


val: Scanning /data/konrad/workspace/datasets/yolo_detection_splt/val/labels... 1379 images, 0 backgrounds, 0 corrupt: 1

val: New cache created: /data/konrad/workspace/datasets/yolo_detection_splt/val/labels.cache


...

val: Scanning /data/konrad/workspace/datasets/yolo_detection_splt/test/labels... 283 images, 0 backgrounds, 0 corrupt: 1

val: New cache created: /data/konrad/workspace/datasets/yolo_detection_splt/test/labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:02<


                   all        283        283      0.352       0.72      0.459      0.361
              Diarrhea         99         99      0.411      0.606      0.572      0.454
             Pneumonia         75         75      0.246      0.773      0.262      0.207
               Healthy        109        109        0.4       0.78      0.543      0.422
Speed: 0.2ms preprocess, 3.0ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to /data/konrad/workspace/training_log/yolo_detect72
Ultralytics YOLOv8.2.58 🚀 Python-3.10.12 torch-2.3.1+cu121 CPU (AMD Ryzen Threadripper PRO 5975WX 32-Cores)

PyTorch: starting from '/data/konrad/workspace/training_log/yolo_detect7/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 7, 8400) (49.6 MB)

TorchScript: starting export with torch 2.3.1+cu121...
TorchScript: export success ✅ 1.4s, saved as '/data/konrad/workspace/training_log/yolo_detect7/weights/best.torchscript' (99.1 MB)

Export complete (3.1s)
Res

'/data/konrad/workspace/training_log/yolo_detect7/weights/best.torchscript'